In [25]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm

from torch.utils.data import TensorDataset

from transformers import (ElectraForSequenceClassification, ElectraTokenizerFast)

from matplotlib import pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import classification_report, confusion_matrix

# Get data

In [26]:
df = pd.read_csv('./../../../labeledTweets/allLabeledTweets.csv')
df = df[['id', 'message', 'label']]
df = df.drop_duplicates()
print(df.shape[0])
df.head()

2020


,id,message,label
0,1478404,Tiek vērtēti trīs potenciālie airBaltic invest...,0
1,1478695,Augulis: #airBaltic “potenciālie pircēji ir no...,0
2,1478812,airBaltic uzsāks lidojumus uz diviem jauniem g...,0
3,1479295,Ministrs: Sarunas turpinās ar trīs potenciālaj...,0
4,1480097,@krisjaniskarins @Janis_Kazocins @EU2017EE Net...,0


In [27]:
df['label'].value_counts()

0    965
2    645
1    410
Name: label, dtype: int64

In [28]:
newLine ="\\n|\\r"
urls = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
numbers = '\d+((\.|\-)\d+)?'
mentions = '\B\@([\w\-]+)'
hashtag = '#'
whitespaces = '\s+'
leadTrailWhitespace = '^\s+|\s+?$'

df['clean_message'] = df['message']
df['clean_message'] = df['clean_message'].str.replace(newLine,' ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(urls,' URL ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(mentions,' MENTION ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(numbers,' NMBR ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(hashtag,' ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(whitespaces,' ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(leadTrailWhitespace,'',regex=True)

df.head()

,id,message,label,clean_message
0,1478404,Tiek vērtēti trīs potenciālie airBaltic invest...,0,Tiek vērtēti trīs potenciālie airBaltic invest...
1,1478695,Augulis: #airBaltic “potenciālie pircēji ir no...,0,Augulis: airBaltic “potenciālie pircēji ir no ...
2,1478812,airBaltic uzsāks lidojumus uz diviem jauniem g...,0,airBaltic uzsāks lidojumus uz diviem jauniem g...
3,1479295,Ministrs: Sarunas turpinās ar trīs potenciālaj...,0,Ministrs: Sarunas turpinās ar trīs potenciālaj...
4,1480097,@krisjaniskarins @Janis_Kazocins @EU2017EE Net...,0,MENTION MENTION MENTION Netrāpīs kādam AirBalt...


# Train, validate split (balanced)

In [29]:
df_0 = df[df['label']==0]
df_1 = df[df['label']==1]
df_2 = df[df['label']==2]

trainLabelSize = round(df_1.shape[0]*0.85)
trainLabelSize

348

In [30]:
df_0 = df_0.sample(trainLabelSize, random_state=42)
df_1 = df_1.sample(trainLabelSize, random_state=42)
df_2 = df_2.sample(trainLabelSize, random_state=42)

df_train = pd.concat([df_0, df_1, df_2])
# Shuffle rows
df_train = sklearn.utils.shuffle(df_train, random_state=42)

df_train['label'].value_counts()

0    348
1    348
2    348
Name: label, dtype: int64

In [31]:
df_val = df.merge(df_train, on=['id', 'message', 'label', 'clean_message'], how='left', indicator=True)
df_val = df_val[df_val['_merge']=='left_only']
df_val = df_val[['id', 'message', 'label', 'clean_message']]
df_val['label'].value_counts()

0    617
2    297
1     62
Name: label, dtype: int64

# Tokenizer "google/electra-base-discriminator"

In [32]:
tokenizer = ElectraTokenizerFast.from_pretrained('google/electra-base-discriminator', do_lower_case=False)

## Find popular UNK tokens

In [33]:
unk_tokens = []
for message in df.clean_message.values:
    list_of_space_separated_pieces = message.strip().split()
    ids = [tokenizer(piece, add_special_tokens=False)["input_ids"] for piece in list_of_space_separated_pieces]
    unk_indices = [i for i, encoded in enumerate(ids) if tokenizer.unk_token_id in encoded]
    unknown_strings = [piece for i, piece in enumerate(list_of_space_separated_pieces) if i in unk_indices]
    for unk_str in unknown_strings:
        unk_tokens.append(unk_str)
        
import collections

counter=collections.Counter(unk_tokens)
print(counter.most_common(100))

most_common_values= [word for word, word_count in counter.most_common(100)]

[('MENTION', 2353), ('URL', 951), ('NMBR', 897), ('arī', 202), ('kā', 173), ('Sveiki!', 134), ('Tele', 116), ('Swedbank', 109), ('Latvijas', 107), ('pēc', 105), ('Rimi', 102), ('lūdzu,', 98), ('SEB', 95), ('LMT', 92), ('Ja', 86), ('Sveiks!', 82), ('Vai', 78), ('Labdien!', 77), ('mūsu', 74), ('pieslēguma', 70), ('vēl', 68), ('DM', 68), ('Kate', 68), ('tā', 67), ('būs', 59), ('DM.', 59), ('Maxima', 57), ('šo', 53), ('līdz', 53), ('Latvenergo', 50), ('Un', 47), ('Paldies', 46), ('DM,', 45), ('ļoti', 44), ('Man', 44), ('informāciju', 44), ('airBaltic', 42), ('Skaties', 41), ('tieši', 40), ('Bet', 40), ('atsūti', 40), ('TET', 39), ('varētu', 35), ('Jūsu', 35), ('Lai', 35), ('Jums', 35), ('Atsūti,', 35), ('Es', 34), ('Kā', 34), ('kurā', 34), ('Tet', 34), ('precīzu', 34), ('vairāk', 33), ('Tā', 33), ('laikā', 33), ('G', 33), ('veikalā', 32), ('būtu', 32), ('Paldies,', 31), ('Maximā', 31), ('nekā', 30), ('Tas', 30), ('Rīgas', 29), ('šeit', 29), ('Sveiks,', 29), ('Aicinām', 29), ('RIMI', 28), (

In [34]:
tokenizer.add_tokens(most_common_values, special_tokens=True)

100

### Find max length for tokenizer

In [35]:
token_lens = []
for txt in list(df.clean_message.values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    
max_length = max(token_lens)
max_length

119

### Encode messages

In [36]:
encoded_data_train = tokenizer.batch_encode_plus(
    df_train["clean_message"].values.tolist(), 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',
    truncation=True,
    max_length=max_length, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df_val["clean_message"].values.tolist(), 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',
    truncation=True,
    max_length=max_length, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df_train.label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_val.label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

len(dataset_train), len(dataset_val)

(1044, 976)

# Model "google/electra-base-discriminator"

In [48]:
model = ElectraForSequenceClassification.from_pretrained("google/electra-base-discriminator",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.d

In [49]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30622, 768)

In [50]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
dataloader_validation = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=batch_size)

In [51]:
from transformers import get_linear_schedule_with_warmup

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [52]:
epochs = 5
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

In [53]:
# Function to measure weighted F1
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [54]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model.to(device)
print(device)

cpu


In [55]:
# Function to evaluate model. Returns average validation loss, predictions, true values
def evaluate(dataloader_val):
    model.eval()
    loss_val_total = 0
    predictions, true_vals = [], []
    
    progress_bar = tqdm(dataloader_val, desc='Validating:', leave=False, disable=False)
    for batch in progress_bar:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

# Evaluate untrained model

In [46]:
_, predictions, true_vals = evaluate(dataloader_validation)

from sklearn.metrics import classification_report, confusion_matrix

preds_flat = np.argmax(predictions, axis=1).flatten()

print(classification_report(true_vals, preds_flat))
print(f1_score_func(predictions, true_vals))
pd.DataFrame(confusion_matrix(true_vals, preds_flat),
        index = [['actual', 'actual', 'actual'], ['neutral', 'positive', 'negative']],
        columns = [['predicted', 'predicted', 'predicted'], ['neutral', 'positive', 'negative']])

Validating::   0%|          | 0/31 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.44      0.02      0.04       617
           1       0.07      0.16      0.09        62
           2       0.33      0.88      0.48       297

    accuracy                           0.29       976
   macro avg       0.28      0.35      0.20       976
weighted avg       0.38      0.29      0.17       976

0.1744848616656692


predicted                  
                  neutral positive negative
actual neutral         12      119      486
       positive         0       10       52
       negative        15       21      261

# Train

In [56]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    torch.save(model.state_dict(), f'modelsUNK/finetuned_ELECTRAbase_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    
    preds_flat = np.argmax(predictions, axis=1).flatten()
    
    print('Classification report:')
    print(classification_report(true_vals, preds_flat))
    print('Confusion matrix:')
    print(pd.DataFrame(confusion_matrix(true_vals, preds_flat),
            index = [['actual', 'actual', 'actual'], ['neutral', 'positive', 'negative']],
            columns = [['predicted', 'predicted', 'predicted'], ['neutral', 'positive', 'negative']]))

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/33 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.0921873251597087


Validating::   0%|          | 0/31 [00:00<?, ?it/s]

Validation loss: 1.0915642246123283
F1 Score (Weighted): 0.4951391201662471
Classification report:
              precision    recall  f1-score   support

           0       0.75      0.41      0.53       617
           1       0.08      0.18      0.11        62
           2       0.40      0.67      0.50       297

    accuracy                           0.48       976
   macro avg       0.41      0.42      0.38       976
weighted avg       0.60      0.48      0.50       976

Confusion matrix:
                predicted                  
                  neutral positive negative
actual neutral        255       94      268
       positive        21       11       30
       negative        66       33      198


...

Validation loss: 1.0461630628954979
F1 Score (Weighted): 0.594702431314866
Classification report:
              precision    recall  f1-score   support

           0       0.82      0.55      0.66       617
           1       0.08      0.15      0.11        62
           2       0.47      0.72      0.57       297

    accuracy                           0.57       976
   macro avg       0.46      0.47      0.44       976
weighted avg       0.66      0.57      0.59       976

Confusion matrix:
                predicted                  
                  neutral positive negative
actual neutral        339       68      210
       positive        22        9       31
       negative        54       30      213


Epoch 5:   0%|          | 0/33 [00:00<?, ?it/s]


Epoch 5
Training loss: 1.0244022770361467


Validating::   0%|          | 0/31 [00:00<?, ?it/s]

Validation loss: 1.0690233053699616
F1 Score (Weighted): 0.5673044929373887
Classification report:
              precision    recall  f1-score   support

           0       0.84      0.47      0.60       617
           1       0.11      0.23      0.14        62
           2       0.46      0.77      0.58       297

    accuracy                           0.55       976
   macro avg       0.47      0.49      0.44       976
weighted avg       0.68      0.55      0.57       976

Confusion matrix:
                predicted                  
                  neutral positive negative
actual neutral        291       91      235
       positive        17       14       31
       negative        39       28      230


# Evaluate best model

In [210]:
model.load_state_dict(torch.load('modelsBase/finetuned_ELECTRAbase_epoch_X.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
preds_flat = np.argmax(predictions, axis=1).flatten()

In [ ]:
print(f1_score_func(predictions, true_vals))
print(classification_report(true_vals, preds_flat))

In [ ]:
pd.DataFrame(confusion_matrix(true_vals, preds_flat),
        index = [['actual', 'actual', 'actual'], ['neutral', 'positive', 'negative']],
        columns = [['predicted', 'predicted', 'predicted'], ['neutral', 'positive', 'negative']])